In [106]:
import pandas as pd

df = pd.read_csv('../../../data/final/dtdata.csv', index_col=0)

In [107]:
new_column_names = ['seccio', 'altmax', 'amplmax', 'perime', 'Longitud', 'Velocitat',
                   'AltAigua', 'Cabal', 'seccio_1', 'altmax_1', 'amplmax_1', 'perime_1',
                   'Longitud_1', 'Velocitat_1', 'AltAigua_1', 'Cabal_1', 'seccio_2',
                   'altmax_2', 'amplmax_2', 'perime_2', 'Longitud_2', 'Velocitat_2',
                   'AltAigua_2', 'Cabal_2', 'seccio_3', 'altmax_3', 'amplmax_3',
                   'perime_3', 'Longitud_3', 'Velocitat_3', 'AltAigua_3', 'Cabal_3',
                   'seccio_4', 'altmax_4', 'amplmax_4', 'perime_4', 'Longitud_4',
                   'Velocitat_4', 'AltAigua_4', 'Cabal_4', 'seccio_5', 'altmax_5',
                   'amplmax_5', 'perime_5', 'Longitud_5', 'Velocitat_5', 'AltAigua_5',
                   'Cabal_5', 'barri']

# Get the current column names
current_column_names = df.columns

# Replace the names of the last 49 columns
df.columns = current_column_names[:-49].tolist() + new_column_names
df.drop('value_2', axis=1, inplace=True)

In [109]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.base import clone
from sklearn.datasets import make_classification

# Training a tree with only the features of the section to be predicted

In [234]:
X = df.drop(df.filter(regex=("(_[0-5]_[0-5]|_[2-5])")).columns, 
        axis=1)
y= X['value_y']
X.drop(['amount_rain_std_1', 'amount_rain_mean_1', 'altmax_1',
        'Longitud_1', 'Velocitat_1', 'Cabal_1', 'value_y',
        'seccio_1', 'amplmax_1', 'perime_1', 'AltAigua_1'],
       axis=1, inplace=True)

In [235]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [239]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score

# Define the hyperparameter space
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'max_features': hp.uniform('max_features', 0.65, 1),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(3, 13)),
    'min_samples_split': hp.choice('min_samples_split', range(10, 25))
}

# Objective function
def objective(params):
    classifier = DecisionTreeClassifier(**params)
    score = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
    return {'loss': -score, 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

mapped_best = {
    'max_depth': best['max_depth'] + 3,  # because range starts from 3
    'max_features': best['max_features'],
    'min_samples_leaf': best['min_samples_leaf'] + 3,  # because range starts from 3
    'min_samples_split': best['min_samples_split'] + 10  # because range starts from 10
}

print(mapped_best)


100%|████████████| 50/50 [00:00<00:00, 61.96trial/s, best loss: -0.6378706350611265]
{'max_depth': 5, 'max_features': 0.9937901571090159, 'min_samples_leaf': 12, 'min_samples_split': 24}


In [242]:
best_classifier = DecisionTreeClassifier(**{'max_depth': 7, 'max_features': 0.794612916790575, 'min_samples_leaf': 12, 'min_samples_split': 19, 'random_state': 4})
best_classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, max_features=0.794612916790575,
                       min_samples_leaf=12, min_samples_split=19)

In [243]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Predict using the best model
y_pred_test_expanded = best_classifier.predict(X_test)
y_prob_test_expanded = best_classifier.predict_proba(X_test)[:, 1]

# Calculate the performance metrics for the test set
test_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_test_expanded),
    'Precision': precision_score(y_test, y_pred_test_expanded),
    'Recall': recall_score(y_test, y_pred_test_expanded),
    'F1 Score': f1_score(y_test, y_pred_test_expanded),
    'ROC-AUC': roc_auc_score(y_test, y_prob_test_expanded)
}

test_metrics


{'Accuracy': 0.7486910994764397,
 'Precision': 0.746031746031746,
 'Recall': 0.5949367088607594,
 'F1 Score': 0.6619718309859155,
 'ROC-AUC': 0.8067924954792045}

best:
```
{'max_depth': 7, 'max_features': 0.794612916790575, 'min_samples_leaf': 12, 'min_samples_split': 19}

{'Accuracy': 0.7486910994764397,
 'Precision': 0.746031746031746,
 'Recall': 0.5949367088607594,
 'F1 Score': 0.6619718309859155,
 'ROC-AUC': 0.8067924954792045}
 ```

In [244]:
feature_importance_dict = dict(zip(X_test.columns, best_classifier.feature_importances_))

# Sort the dictionary based on importances
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

sorted_feature_importance

[('value_0', 0.25788450921040773),
 ('amount_rain_std', 0.15887078718324263),
 ('value_1', 0.13788963643114127),
 ('AltAigua', 0.09710192928642877),
 ('altmax', 0.09625560731922002),
 ('perime', 0.09461655442118112),
 ('Velocitat', 0.0516787840041893),
 ('amount_rain_mean', 0.042936260163940115),
 ('Cabal', 0.025239962317264466),
 ('Longitud', 0.019932951206285857),
 ('amplmax', 0.01759301845669885),
 ('cleaning_applied_0', 0.0),
 ('cleaning_applied_1', 0.0),
 ('seccio', 0.0),
 ('barri', 0.0)]

In [245]:
import dtreeviz

viz = dtreeviz.model(
    best_classifier,
    X,
    y,
    target_name="value_y",  # Use 'value_y' as the target variable name
    feature_names=X.columns,
    class_names=y.unique().astype(str)
)


In [248]:
v = viz.view(fancy=True)  
v.show()               
v.save("best_tree_without_neighbours.svg")

# Best possible decision tree

The idea is to get the best possible tree, hyperparameter and feature - wise.

In [213]:
X = df.drop(columns=['value_y'])
y = df['value_y']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [214]:
from sklearn.feature_selection import RFE
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Define the hyperparameter space
space = {
    'max_depth': hp.choice('max_depth', range(3, 20)),
    'max_features': hp.uniform('max_features', 0.65, 0.95),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(3, 20)),
    'min_samples_split': hp.choice('min_samples_split', range(10, 35))
}

def objective(params, X_train_sub, y_train, X_test_sub, y_test):
    classifier = DecisionTreeClassifier(**params)
    classifier.fit(X_train_sub, y_train)
    preds = classifier.predict(X_test_sub)
    score = f1_score(y_test, preds)
    return {'loss': -score, 'status': STATUS_OK}

best_f1_global = float('-inf')
best_features_global = None
best_hyperparameters_global = None

# Apply RFE to get the ranking of features
estimator = DecisionTreeClassifier()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train, y_train)
rankings = selector.ranking_

# Sort features by their ranking
sorted_features = [f for _, f in sorted(zip(rankings, features))]

# Iterate over features based on their ranking
for i in range(1, len(sorted_features) + 1):
    selected_features = sorted_features[:i]

    # Bayesian optimization for the current feature combination
    trials = Trials()
    best_hyperparams = fmin(fn=lambda params: objective(params, X_train[selected_features], y_train, X_test[selected_features], y_test),
                            space=space, algo=tpe.suggest, max_evals=50, trials=trials, verbose=False)

    # Check the best F1 score for the current feature combination
    best_f1_current = -trials.best_trial['result']['loss']

    if best_f1_current > best_f1_global:
        best_f1_global = best_f1_current
        best_features_global = selected_features
        best_hyperparameters_global = best_hyperparams

print("Best F1 Score:", best_f1_global)
print("Best Features:", best_features_global)
print("Best Hyperparameters:", best_hyperparameters_global)


Best F1 Score: 0.7581699346405228
Best Features: ['amount_rain_std', 'value_1_5', 'value_1_3', 'value_1_2', 'altmax', 'value_1', 'Velocitat', 'amount_rain_mean_1', 'Longitud_1']
Best Hyperparameters: {'max_depth': 16, 'max_features': 0.9360168327534356, 'min_samples_leaf': 5, 'min_samples_split': 11}


In [215]:
mapped_best = {
    'max_depth': best_hyperparameters_global['max_depth'] + 3,  # because range starts from 3
    'max_features': best_hyperparameters_global['max_features'],
    'min_samples_leaf': best_hyperparameters_global['min_samples_leaf'] + 3,  # because range starts from 3
    'min_samples_split': best_hyperparameters_global['min_samples_split'] + 10  # because range starts from 10
}

In [216]:
best_classifier = DecisionTreeClassifier(**{'max_depth': 16, 'max_features': 0.9360168327534356, 'min_samples_leaf': 5, 'min_samples_split': 11, 'random_state': 4})
best_classifier.fit(X_train[best_features_global], y_train)

DecisionTreeClassifier(max_depth=19, max_features=0.9360168327534356,
                       min_samples_leaf=8, min_samples_split=21)

In [217]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Predict using the best model
y_pred_test_expanded = best_classifier.predict(X_test[best_features_global])
y_prob_test_expanded = best_classifier.predict_proba(X_test[best_features_global])[:, 1]

# Calculate the performance metrics for the test set
test_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_test_expanded),
    'Precision': precision_score(y_test, y_pred_test_expanded),
    'Recall': recall_score(y_test, y_pred_test_expanded),
    'F1 Score': f1_score(y_test, y_pred_test_expanded),
    'ROC-AUC': roc_auc_score(y_test, y_prob_test_expanded)
}

test_metrics

{'Accuracy': 0.774869109947644,
 'Precision': 0.725,
 'Recall': 0.7341772151898734,
 'F1 Score': 0.7295597484276729,
 'ROC-AUC': 0.8261188969258589}

```
['amount_rain_std', 'value_1_5', 'value_1_3', 'value_1_2', 'altmax', 'value_1', 'Velocitat', 'amount_rain_mean_1', 'Longitud_1']

{'max_depth': 16, 'max_features': 0.9360168327534356, 'min_samples_leaf': 5, 'min_samples_split': 11}

{'Accuracy': 0.774869109947644,
 'Precision': 0.725,
 'Recall': 0.7341772151898734,
 'F1 Score': 0.7295597484276729,
 'ROC-AUC': 0.8261188969258589}
 ```

In [230]:
feature_importance_dict = dict(zip(best_features_global, best_classifier.feature_importances_))

# Sort the dictionary based on importances
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

sorted_feature_importance

[('value_1_5', 0.26785628202856754),
 ('amount_rain_std', 0.16483097142123013),
 ('value_1_2', 0.16466456242988545),
 ('altmax', 0.09673028784446126),
 ('value_1_3', 0.08205223036018434),
 ('Velocitat', 0.07888467622901407),
 ('value_1', 0.06212096604696657),
 ('amount_rain_mean_1', 0.05953181801855086),
 ('Longitud_1', 0.023328205621139762)]

In [226]:
import dtreeviz

viz = dtreeviz.model(
    best_classifier,
    X[best_features_global],
    y,
    target_name="value_y",  # Use 'value_y' as the target variable name
    feature_names=best_features_global,
    class_names=y.unique().astype(str)
)


In [227]:
v = viz.view(fancy=True)  
v.show()               
v.save("best_tree.svg")

/Users/marc.ribalta/anaconda3/envs/phd/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [228]:
v = viz.view(fancy=False)
v.show()               
v.save("best_tree_unfancy.svg")